In [94]:
# Enable auto-reload of modified python files.
%load_ext autoreload
%autoreload 2
%matplotlib qt 

# imports
import settings
import warnings
import os
import re as re
from matplotlib.pyplot import ginput
import matplotlib.pyplot as plt
import PIL
import pandas as pd

warnings.filterwarnings('ignore')

def clean_image_name(filename):
    return re.findall(r"(.*)\.", filename)[0]

def list_images(IMG_DIRECTORY,img_type):
    image_list = []
    for filename in os.listdir(IMG_DIRECTORY):
        if filename.endswith(img_type): 
            if "apr" in filename:
                if 'false' not in str(filename):
                    if 'practice' not in str(filename):
                        image_list.append(clean_image_name(filename))
    return image_list

def set_directory(condition):
    return settings.DRAWINGS_PATH + '/' + str(condition) + '/'

def load_corners_csv(data_path):
    return pd.read_csv(data_path)

def get_drawing_IDs(conditions):
    drawings = []
    for condition in conditions:
        drawing_dir = set_directory(condition)
        drawings.extend(list_images(drawing_dir, '.jpg'))
    return drawings

def calc_num_unrated(drawing_path, data_path):
    conditions = [name for name in os.listdir(drawing_path) if os.path.isdir(os.path.join(drawing_path, name))]
    drawings = get_drawing_IDs(conditions)
    corner_df = load_corners(data_path)
    unrated = []
    for drawing in drawings:
        if corner_df['ID'].str.contains(drawing).any() == False:
            unrated.append(drawing)
    print(len(unrated), ' of ', len(drawings), 'drawings left unrated')
    return unrated

def find_corners(drawing):
    corners = []
    condition = 'search'
    if 'C' in drawing:
        condition = 'memorize'
    drawing_dir = set_directory(condition)
   
    img = PIL.Image.open(str(drawing_dir + drawing + '.jpg'))

    # show image
    fig, ax= plt.subplots()
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()
    ax.imshow(img)
    corners.append(plt.ginput(4,0))
    df = pd.DataFrame(corners)
    df['ID'] = drawing
    df['condition'] = condition
    df.columns = ['0','1','2','3','ID','condition']
    plt.close("all")
    return df
   
data_path = settings.DATA_PATH + '/corners.csv'

if not os.path.isfile(data_path):
    df = pd.DataFrame(columns=['0','1','2','3','ID','condition'])
    df.to_csv(data_path)

print('label order: top left, top right, bottom left, bottom right')

unrated = num_unrated(settings.DRAWINGS_PATH, data_path)
dfs = []
for drawing in unrated:
    cdf = find_corners(drawing)
    dfs.append(cdf)
    new = pd.concat(dfs)
    old = load_old_corners(data_path)
    corners = pd.concat([old, new])
    corners = corners.reset_index(drop=True)
    corners.to_csv(data_path, index=False)

unrated = num_unrated(settings.DRAWINGS_PATH, data_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
label order: top left, top right, bottom left, bottom right
1  of  518 drawings left unrated
0  of  518 drawings left unrated
